# Forecasting 

![extrapolating](https://imgs.xkcd.com/comics/extrapolating.png)

In this lesson, we will practice forecasting using the following methods:  

- Last observed value  
- Simple average  
- Moving average  
- Holt's Linear Trend  
- Previous cycle  

______________________________


We will walk through steps from previous lessons to get the data ready to model

- Acquire data: prepare.acquire_store_data()  
- Prepare data: prepare.prep_store_data()  
- Split data: prepare.split_store_data()  

Then we will forecast and evaluate using each method. 

In [1]:
import numpy as np
import pandas as pd

#viz
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#for presentation purposes
import warnings
warnings.filterwarnings("ignore")

#working with dates
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt 

#evaluated performance using rmse
import statsmodels.api as sm

#holts linear trend model
from statsmodels.tsa.api import Holt

## Wrangle


In [2]:
df = pd.read_csv('store_item_demand.csv')
df.head().T

,0,1,2,3,4
sale_id,1,2,3,4,5
sale_date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05
store_id,1,1,1,1,1
item_id,1,1,1,1,1
sale_amount,13,11,14,13,10
item_brand,Riceland,Riceland,Riceland,Riceland,Riceland
item_name,Riceland American Jazmine Rice,Riceland American Jazmine Rice,Riceland American Jazmine Rice,Riceland American Jazmine Rice,Riceland American Jazmine Rice
item_price,0.84,0.84,0.84,0.84,0.84
store_address,12125 Alamo Ranch Pkwy,12125 Alamo Ranch Pkwy,12125 Alamo Ranch Pkwy,12125 Alamo Ranch Pkwy,12125 Alamo Ranch Pkwy
store_zipcode,78253,78253,78253,78253,78253



1. sale_date to datetime
2. sort values by date
3. set index
4. new field: dollars_sold = sale_amount * item_price
5. rename sale_amount to items_sold to make the two columns easier to understand what the data represents. 
6. resample daily (The original granularity is daily, but there are multiple records of the same days across multiple stores.)
7. remove leap days!


We will resample to daily, but essentially what we are doing is grouping by the day and aggregating using sum. The original granularity is daily, but there are multiple records of the same days across multiple stores. 

In [ ]:
#how maggie writes a function:
#convert the sale date to datetime format
df['ds'] = pd.to_datetime(df.sale_date)

In [ ]:
#sort values by date
df.sort_values('ds').tail()

In [ ]:
#set index
df = df.set_index('ds')

In [ ]:
df['dollars_sold'] = df.sale_amount * df.item_price

In [ ]:
df[['dollars_sold', 'sale_amount', 'item_price']].head()
df[['dollars_sold', 'sale_amount', 'item_price']].describe()

In [ ]:
# rename sale amount ot items sold and drop original column of sale amount
df = df.assign(item_sold = df.sale_amount).drop(columns=['sale_amount'])

In [ ]:
df.head()

In [ ]:
#aggregate or resample daily by summing dollars and items sold
df_resampled = df.resample('d')[['dollars_sold', 'item_sold']].sum()

In [ ]:
#remove leap day
df_resampled[df_resampled.index != '2016-02-29' ]

In [ ]:
def prep_data(df):
    """
    This function takes in a dataframe and returnsa dataframe with
    the dates as index, aggregated by summing at the daily level,
    changing the sale amount column name to be item sold, creating
    a new feature of dollars_sold which is sale_amount * item_price
    and removing the leap day.
    """
    return (df.assign(ds = pd.to_datetime(df.sale_date)).
            # sort values by date
            sort_values('ds').
            # create dollars_sold column
            assign(dollars_sold = df.sale_amount * df.item_price).
            # sale_amount to be items_sold
            assign(items_sold = df.sale_amount).
            # aggregate daily by summing the values
            groupby(['ds'])[['dollars_sold', 'items_sold']].sum().
            # set index to date
            reset_index().set_index('ds')
           )

In [ ]:
df = prep_data(df)

In [ ]:
df.head()

In [ ]:
# remove leap days

df = df[df.index != '2016-02-29']


## Split

1. We will use the training proportion method to split.    
2. Identify the total length of the dataframe and multiply by `train_prop` to get the number of rows that equates to the first x% of the dataframe, which equates to the first x% of the time covered in the data.   (`x = train_prop * 100`)  
3. Select row indices from 0 up to the index representing x-percentile for train, and from the index representing x-percentile through the end of the dataframe for test. In both of these, we will reset the index in order to return dataframes sorted by datetime.  
4. Return train and test dataframes.  

In [ ]:
train_size = int(len(df) * .5)
train_size

In [ ]:
validate_size = int(len(df) * .3)
validate_size

In [ ]:
test_size = int(len(df) - train_size - validate_size)
test_size

In [ ]:
validate_end_index = train_size + validate_size
validate_end_index

Use those values to split our dataframe

In [ ]:
train = df[: train_size]
validate = df[train_size:validate_end_index]
test = df[validate_end_index:]

**Verify Splits**

Does the length of each df equate to the length of the original df? 

In [ ]:
print(len(train) + len(validate) + len(test) == len(df))

Does the first row of original df equate to the first row of train? 

In [ ]:
print(df.head(1) == train.head(1))

Is the last row of train the day before the first row of validate? And the same for validate to test? 

In [ ]:
pd.concat([train.tail(1), validate.head(1)])


In [ ]:
pd.concat([validate.tail(1), test.head(1)])

Is the last row of test the same as the last row of our original dataframe? 

In [ ]:
pd.concat([test.tail(1), df.tail(1)])

Let's plot our data first, viewing where the data is split into train and test. 

In [ ]:
for col in train.columns:
    plt.figure(figsize=(12,4))
    plt.plot(train[col])
    plt.plot(validate[col])
    plt.plot(test[col])
    plt.ylabel(col)
    plt.title(col)
    plt.show()

Before we try out different methods for forecasting sales and number of items sold, let's create a couple of functions that will be helpful in evaluating each of the methods that follow. 

`evaluate()` will compute the Mean Squared Error and the Rood Mean Squared Error to evaluate.  

In [ ]:
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse

`plot_and_eval()` will use the evaluate function and also plot train and test values with the predicted values in order to compare performance. 

In [ ]:
def plot_and_eval(target_var):
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label='Train', linewidth=1)
    plt.plot(validate[target_var], label='Validate', linewidth=1)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

Write `append_eval_df(model_type)` to append evaluation metrics for each model type, target variable, and metric type, along with the metric value into our `eval_df` data frame object. Which we will create an empty `eval_df` dataframe object to start. 

In [ ]:
# create an empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

# function to store the rmse so that we can compare
def append_eval_df(model_type, target_var):
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var],
        'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

In [ ]:
eval_df

## Forecast 

Forecasting is another word for predicting time series data. 

1. Last Observed Value
2. Simple Average
3. Moving Average
4. Holt's Linear Trend
5. Previous Cycle


### Last observed value

The simplest method for forecasting is to predict all future values to be the last observed value.  

**Make Predictions**

Dollars

In [ ]:
dollars = round(train['dollars_sold'][-1:][0], 2)
dollars

Items

In [ ]:
items = train['items_sold'][-1:][0]
items

In [ ]:
yhat_df = pd.DataFrame({'dollars_sold': [dollars], 
                        'items_sold': [items]}, 
                      index = validate.index)

yhat_df.head()
yhat_df.describe()

You can see, when peeking into yhat_df, that every predicted value is the same.  

**Plot Actual vs. Predicted Values**

Now, let's plot actual and predicted values

In [ ]:
plot_and_eval('dollars_sold')

In [ ]:
for col in train.columns:
    plot_and_eval(col)

**Evaluate** 

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'last_observed_value', 
                             target_var = col)

eval_df

### Simple Average

Take the simple average of historical values and use that value to predict future values.   

This is a good option for an initial baseline. Every future datapoint (those in 'test') will be assigned the same value, and that value will be the overall mean of the values in train. 

**Make Predictions**

Dollars: establishing the value of the prediction we will make

In [ ]:
# compute simple average

# plt.plot(train['dollars_sold'])
dollars = round(train['dollars_sold'].mean(),2)
dollars

Items: establishing the value of the prediction we will make

In [ ]:
items = round(train['items_sold'].mean(),2)
items

Apply predictions to our observations

In [ ]:
def make_predictions():
    yhat_df = pd.DataFrame({'dollars_sold': [dollars],
                           'items_sold': [items]}, 
                           index = validate.index)
    return yhat_df


In [ ]:
yhat_df = make_predictions()

In [ ]:
yhat_df.head()

In [ ]:
yhat_df.describe()

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Simple Average**

In [ ]:
for col in train.columns:
    plot_and_eval(col)

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type='simple_average', 
                            target_var = col)
eval_df

### Moving Average

In this example, we will use a 30-day moving average to forecast. In other words, the average over the last 30-days will be used as the forecasted value. 

In [ ]:
# train['dollars_sold'].tail(30)
plt.figure(figsize=(12,4))
plt.plot(train['dollars_sold'].rolling(7).mean())
plt.plot(train['dollars_sold'].rolling(30).mean())
plt.plot(train['dollars_sold'].rolling(90).mean())
plt.plot(train['dollars_sold'].rolling(120).mean())
plt.plot(train['dollars_sold'], alpha=.3)

In [ ]:
# demonstrate that the mean of the first 30 days 
# is equal to rolling(30) on day 30

print(train['dollars_sold'].rolling(30).mean()[29])
print(train['dollars_sold'].head(30).mean())

**Make Predictions**

In [ ]:
period = 30 

dollars = round(train['dollars_sold'].rolling(period).mean()[-1], 2)
items = round(train['items_sold'].rolling(period).mean()[-1], 2)

print(dollars, items)

In [ ]:
yhat_df = make_predictions()
yhat_df.head()

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Moving Average**

In [ ]:
for col in train.columns:
    plot_and_eval(col)

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = '30d_moving_avg', 
                            target_var = col)

eval_df

Let's try out several other values for periods:

In [ ]:
periods = [4, 12, 26, 52, 104]

for p in periods: 
    dollars = round(train['dollars_sold'].rolling(p).mean()[-1], 2)
    items = round(train['items_sold'].rolling(p).mean()[-1],2)
    yhat_df = make_predictions()
    model_type = str(p) + 'd_moving_avg'
    for col in train.columns:
        eval_df = append_eval_df(model_type = model_type, 
                                 target_var = col)

In [ ]:
eval_df

Which is best so far? 

In [ ]:
min_items_rmse = eval_df[eval_df.target_var == 'items_sold']['rmse'].min()

eval_df[eval_df.rmse == min_items_rmse]

### Holt's Linear Trend


Exponential smoothing applied to both the average and the trend (slope).  

- $\alpha$ / smoothing_level: smoothing parameter for mean. Values closer to 1 will have less of a smoothing effect and will give greater weight to recent values.   
- $\beta$ / smoothing_slope: smoothing parameter for the slope. Values closer to 1 will give greater weight to recent slope/values. 



**Seasonal Decomposition**


First, let's take a look at the seasonal decomposition for each target. 

In [ ]:
import statsmodels.api as sm

In [ ]:
for col in train.columns:
    print(col, '\n')
    sm.tsa.seasonal_decompose(train[col].resample('W').mean()).plot()
    plt.show()

#### Basic Holt's Linear Trend

**Make Predictions**

Now, like we would when using sklearn, we will create the Holt object, fit the model, and make predictions. 

Holt: 

- exponential = True/False (exponential vs. linear growth, additive vs. multiplicative)
- damped $\phi$ = True/False: with Holt, forecasts will increase or decrease indefinitely into the future.  To avoid this, use the Damped trend method which has a damping parameter 0< ϕ <1. 


fit: 

- smoothing_level ($\alpha$): value between (0,1)
- smoothing_slope ($\beta$): value between (0,1)
- optimized: use the auto-optimization that allow statsmodels to automatically find an optimized value for us. 

In [ ]:
for col in train.columns:
    model = Holt(train[col], exponential=False, damped=True)
    model = model.fit(optimized=True)
    yhat_items = model.predict(start = validate.index[0],
                               end = validate.index[-1])
    yhat_df[col] = round(yhat_items, 2)

In [ ]:
yhat_df.head()

**Plot Actual vs. Predicted Values**

In [ ]:
for col in train.columns:
    plot_and_eval(target_var = col)

**Evaluate**

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'holts_optimized', 
                            target_var = col)

In [ ]:
eval_df.sort_values(by='rmse')

Let's do another model, changing some hyperparameters. 


In [ ]:
for col in train.columns:
    model = Holt(train[col], exponential=False)
    model = model.fit(smoothing_level=.1, 
                      smoothing_slope=.1, 
                      optimized=False)
    yhat_items = model.predict(start = validate.index[0],
                               end = validate.index[-1])
    yhat_df[col] = round(yhat_items, 2)

In [ ]:
for col in train.columns:
    plot_and_eval(target_var = col)

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'holts_.1', 
                            target_var = col)
eval_df.sort_values(by='rmse')

### Predict Based on Previous Cycle

Take all the 2016 data points, compute the daily delta, year-over-year, average that delta over all the days, and adding that average to the previous year's value on a day will give you the forecast for that day. 

If a primary cycle is weekly, then you may want to do this on a week-over-week cadence. 

In the below example:  
1. Compute the 365 average year over year differences from 2013 through 2015
2. Add that average delta to the values during 2015. 
3. Set the index in your yhat dataframe to represent the dates those predictions are make for. 

Let's get started....

**Re-split data**

In [ ]:
train = df[:'2015']
validate = df['2016']
test = df['2017']

print(train.shape)
print(validate.shape)
print(test.shape)

train.head()

**Make Predictions**

In [ ]:
# finding the year-over-year difference for each day from 2013 to 2015
# taking the mean, and then adding that value to the daily 2015 values. 

yhat_df = train['2015'] + train.diff(365).mean()

In [ ]:
# let's peek into the prediction we will make for 1/1/2016
# by comparing the predicted value 
# (2015 value + year-over-year average difference)
# to the actual 1/1/2016 value
pd.concat([yhat_df.head(1), validate.head(1)])

In [ ]:
# set yhat_df to index of validate

yhat_df.index = validate.index

yhat_df.describe()

In [ ]:
yhat_df.head()

In [ ]:
yhat_df.shape

**Plot and Evaluate**

In [ ]:
for col in train.columns:
    plot_and_eval(target_var = col)
    eval_df = append_eval_df(model_type = "previous_year", 
                            target_var = col)

## Conclusion

Which model did the best? 

In [ ]:
dollars_min_rmse = eval_df.groupby('target_var')['rmse'].min()[0]

items_min_rmse = eval_df.groupby('target_var')['rmse'].min()[1]

# find which model that is
eval_df[((eval_df.rmse == dollars_min_rmse) | 
         (eval_df.rmse == items_min_rmse))]

Let's test it out on our out-of-sample data

We will be using train + validate to predict test. 

In [ ]:
yhat_df = validate + train.diff(365).mean()
yhat_df.index = test.index

In [ ]:
def final_plot(target_var):
    plt.figure(figsize=(12,4))
    plt.plot(train[target_var], label='train')
    plt.plot(validate[target_var], label='validate')
    plt.plot(test[target_var], label='test')
    plt.plot(yhat_df[target_var], alpha=.5)
    plt.title(target_var)
    plt.show()

In [ ]:
rmse_dollars = sqrt(mean_squared_error(test['dollars_sold'], 
                                       yhat_df['dollars_sold']))

rmse_items = sqrt(mean_squared_error(test['items_sold'], 
                                       yhat_df['items_sold']))

In [ ]:
print('rmse-dollars_sold: ', rmse_dollars)
print('rmse-items_sold: ', rmse_items)

In [ ]:
for col in train.columns:
    final_plot(col)

In [ ]:
# to predict 2018

yhat_df = test + train.diff(365).mean()

yhat_df.index = test.index + pd.Timedelta('1Y')

In [ ]:
yhat_df.head()

In [ ]:
def final_plot(target_var):
    plt.figure(figsize=(12,4))
    plt.plot(train[target_var], label='train')
    plt.plot(validate[target_var], label='validate')
    plt.plot(test[target_var], label='test')
    plt.plot(yhat_df[target_var], alpha=.5)
    plt.title(target_var)
    plt.show()

In [ ]:
for col in train.columns:
    final_plot(col)

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage or store_item_sales

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 
2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:
    - Simple Average
    - Moving Average
    - Holt's Linear Trend Model
    - Based on previous year/month/etc., this is up to you.

Optional: Using store item demand

1. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
2. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
3. Plot the 2018 monthly sales predictions.